In [11]:
import open3d as o3d
import numpy as np
from scipy.spatial import cKDTree
import os


In [13]:
# 파일 경로 설정
mesh_path = "C:/Users/konyang/Desktop/MeshCNN_TF/data/dataset/mesh/A0083_abnormal.obj"
label_path = "C:/Users/konyang/Desktop/MeshCNN_TF/data/dataset/lebel/A0083_abnormal_label.npy"

# 메쉬 불러오기
mesh_o3d = o3d.io.read_triangle_mesh(mesh_path)
mesh_o3d.compute_vertex_normals()

# 라벨 불러오기
labels = np.load(label_path)
print(f"Original Mesh: {len(mesh_o3d.triangles)} faces, Labels: {len(labels)}")


Original Mesh: 359636 faces, Labels: 359636


In [14]:
# 목표 면 수 설정 (정점 5,000개 정도를 목표로)
target_face_count = 10000

# 단순화
simplified_o3d = mesh_o3d.simplify_quadric_decimation(target_number_of_triangles=target_face_count)
print(f"Simplified Mesh: {len(simplified_o3d.triangles)} faces")


Simplified Mesh: 9999 faces


In [15]:
# 원본 메쉬의 면 중심점
original_faces = np.asarray(mesh_o3d.triangles)
original_vertices = np.asarray(mesh_o3d.vertices)
original_centroids = original_vertices[original_faces].mean(axis=1)

# 단순화된 메쉬의 면 중심점
simplified_faces = np.asarray(simplified_o3d.triangles)
simplified_vertices = np.asarray(simplified_o3d.vertices)
simplified_centroids = simplified_vertices[simplified_faces].mean(axis=1)

# 최근접 원본 면 탐색 (KDTree)
tree = cKDTree(original_centroids)
_, idx = tree.query(simplified_centroids)


In [16]:
# 라벨 재배열
simplified_labels = labels[idx]

# 저장 경로 설정
save_dir = r"C:\Users\konyang\Desktop\MeshCNN_TF\data\dataset\label_map"
os.makedirs(save_dir, exist_ok=True)

# 파일 저장
mesh_save_path = os.path.join(save_dir, "A0083_abnormal_simplified.obj")
label_save_path = os.path.join(save_dir, "A0083_abnormal_simplified_label.npy")

o3d.io.write_triangle_mesh(mesh_save_path, simplified_o3d)
np.save(label_save_path, simplified_labels)

print(f"Saved:\n- Mesh: {mesh_save_path}\n- Labels: {label_save_path}")


[Open3D WARNING] Write OBJ can not include triangle normals.
Saved:
- Mesh: C:\Users\konyang\Desktop\MeshCNN_TF\data\dataset\label_map\A0083_abnormal_simplified.obj
- Labels: C:\Users\konyang\Desktop\MeshCNN_TF\data\dataset\label_map\A0083_abnormal_simplified_label.npy


In [19]:
# 단순화된 메쉬 경로
simplified_mesh_path = r"C:\Users\konyang\Desktop\MeshCNN_TF\data\dataset\label_map\A0083_abnormal_simplified.obj"

# 메쉬 불러오기
mesh = o3d.io.read_triangle_mesh(simplified_mesh_path)

# 정점 수와 면 수 출력
num_vertices = len(mesh.vertices)
num_faces = len(mesh.triangles)

print(f"Simplified Mesh - Vertices: {num_vertices}, Faces: {num_faces}")

Simplified Mesh - Vertices: 3676, Faces: 9999
